## Part 1: Preprocessing

In [147]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [148]:
# Determine the number of unique values in each column.
print(attrition_df.nunique())

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64


In [149]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]


In [150]:
# Create a list of at least 10 column names to use as X data
column_names = ['Education',
                'Age',
                'DistanceFromHome',
                'JobSatisfaction',
                'OverTime',
                'StockOptionLevel',
                'WorkLifeBalance',
                'YearsAtCompany',
                'YearsSinceLastPromotion',
                'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[column_names]

# Show the data types for X_df
print(X_df.dtypes)

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [151]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=14)

In [152]:
# Convert your X data to numeric data types however you see fit
X_train['OverTime'] = X_train['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)

# Add new code cells as necessary
X_train['OverTime'].value_counts()

,count
OverTime,
0,791
1,311


In [153]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [154]:
# Create a OneHotEncoder for the Department column
ohe_department = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
ohe_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = ohe_department.transform(y_train[['Department']])
y_test_department = ohe_department.transform(y_test[['Department']])

y_train_department

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [155]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = ohe_attrition.transform(y_train[['Attrition']])
y_test_attrition = ohe_attrition.transform(y_test[['Attrition']])

y_train_attrition

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [156]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)


In [157]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu',
                                       name='department_hidden')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department.shape[1],
                                       activation='softmax',
                                       name='department_output')(department_hidden_layer)

In [158]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(32, activation='relu',
                                      name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition.shape[1],
                                      activation='sigmoid',
                                      name='attrition_output')(attrition_hidden_layer)

In [159]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [160]:
# Train the model
history = model.fit(X_train_scaled,
                    {'department_output': y_train_department,
                     'attrition_output': y_train_attrition},
                    epochs=100,
                    batch_size=32,
                    verbose=1)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - attrition_output_accuracy: 0.7431 - department_output_accuracy: 0.5280 - loss: 1.5790
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8354 - department_output_accuracy: 0.6680 - loss: 1.2068
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8310 - department_output_accuracy: 0.6613 - loss: 1.1660
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8529 - department_output_accuracy: 0.6698 - loss: 1.1089
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8419 - department_output_accuracy: 0.6391 - loss: 1.1550
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8338 - department_output_accuracy: 0.6609 - loss: 1.1122
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8473 - department_output_accuracy: 0.6645 - loss: 1.0959
Epoch 8/100
35/35 ━━━━━━━━━

In [161]:
# Evaluate the model with the testing data


In [162]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled,
                              {'department_output': y_test_department,
                               'attrition_output': y_test_attrition,},
                              verbose=1)
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7915 - department_output_accuracy: 0.5241 - loss: 4.4293  


[4.0357584953308105, 0.804347813129425, 0.54076087474823]

In [163]:
# Print the accuracy for both department and attrition
print(f"Department predicions accuracy: {test_results[2]}")
print(f"Attrition predictions accuracy: {test_results[1]}")

Department predicions accuracy: 0.54076087474823
Attrition predictions accuracy: 0.804347813129425


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.  Accuracy isn't always the most reliable way to evaluate a model’s performance. If most employees remain and only a small number leave, the model could simply predict that everyone stays to achieve high accuracy.

A better approach is to assess how well the model identifies the employees who truly leave. Metrics like precision (the proportion of predicted leavers who actually leave) and recall (the proportion of actual leavers correctly predicted) offer more useful insights in this scenario.

.

2. To predict whether an employee will leave (yes or no), I would use the sigmoid function. It outputs a value between 0 and 1, making it ideal for binary decisions.

For determining the best department for an employee, I would apply the softmax function. This function assigns a probability to each department, indicating which one is the most likely fit.

.

3.  Here are several ways to improve the model:

Balance the Data if certain departments have significantly more employees than others, address the imbalance by adding more samples or generating synthetic data to ensure fair training.

Add More Useful Data to generate new features from existing data that could enhance predictive performance.

Fine-Tune Parameters to adjust settings such as the number of layers and neurons to find the optimal configuration.

Combine Models to leverage multiple models and aggregate their predictions to improve overall accuracy.

Prevent Overfitting to apply techniques like dropout (randomly disabling neurons during training) or L2 regularization to avoid the model memorizing the training data.

Conduct Thorough Testing instead of a single data split, use cross-validation by training and testing on different subsets to better assess the model’s performance.

.
